## Import packages

In [8]:
from keras.layers import Dense

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import text
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Flatten,Dropout
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.layers import Input, Conv1D, MaxPooling1D, Concatenate
from keras.optimizers import adam, SGD, rmsprop
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import model_from_json
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
#from feature_selector import FeatureSelector
from sklearn.ensemble import RandomForestRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

## Read, normalize and split traning/test data

In [9]:
#%%df = pd.read_csv('test.csv')
xls = pd.ExcelFile('CA_MO_combined.xlsx')
df1 = pd.read_excel(xls, 'CA_MO_combined_raw')
df2 = pd.read_excel(xls, 'CA_MO_combined_1nm')

dataset = df1.values
# split into input (X) and output (Y) variables, and training/test data
X = dataset[:, 1:202]
X = np.asarray(X,dtype=float)
Y = dataset[:, 205]   # Gs 205; Fs 209;
Y = np.asarray(Y,dtype=float)

#%%
### Module-4: Correlation-based feature selection
#threshold = 0.2 # Pre-defining absolute correlation threshold
#df = s.iloc[:,2:] # extract both y and x features as pd dataframe
#cor = df.corr(method='pearson') # calculate correlation for all variable (method='pearson', method='kendall', method = ‘spearman’)
#cor_target = abs(cor["yield_kg_ha"]) # Extract correlation coefficient only for y with all x input variables
#
#relevant_features = cor_target[cor_target >= threshold] # Selecting highly correlated features based on predefined threshold
#idx = relevant_features.index[1:].tolist() 
#x = s[idx]

#%% normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 0)

num_features = X_train.shape[1]

print (X_train.shape,Y_train.shape,Y_test.shape,X_test.shape, num_features)

#%% Feature Selection

(170, 201) (170,) (73,) (73, 201) 201


## Build DNN layers

In [10]:
#%% Function to create model, required for KerasRegressor
def create_model(learn_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(64, input_dim=num_features, kernel_initializer='normal', activation='relu'))
    model.add(Dense(128, input_dim=num_features, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, input_dim=num_features, activation='relu'))
    model.add(Dense(512, input_dim=num_features, activation='relu'))
    model.add(BatchNormalization())
#    model.add(Dense(512, input_dim=num_features, activation='relu'))
#    model.add(Dense(1024, input_dim=num_features, activation='relu'))
#    model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))
    #model.add(BatchNormalization())
    model.add(Dense(1))

    # Compile model
    opt = adam(lr=learn_rate)
    model.summary()
    model.compile(loss='mse', optimizer= opt, metrics=['mean_squared_error','accuracy'])

    return model

## Run DNN for different batch size, learning and dropout rates

In [11]:
#%% create model
   
model = KerasRegressor(build_fn=create_model, verbose=2)
# grid search epochs, batch size and optimizer
batches = [8,16,24,32]
learn_rates = [0.0005,0.003,0.005,0.007,0.009,0.05,0.07,0.09,0.01]
dropout_rates = [0.20,0.23,0.25,0.27,0.30]

para = {}
for bt in batches:
    for lr in learn_rates:
        for dr in dropout_rates:
            print('batch_size: '+str(bt),'learn_rate: '+str(lr),'dropout_rate: '+str(dr))
            model=create_model(lr, dr)
            
            # CheckPoint
            filepath="weights.best.hdf5"
            checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
            callbacks_list = [checkpoint]
            # Fit the model
            hist = model.fit(X_train, Y_train, epochs=300, batch_size=bt, validation_data = (X_test, Y_test),callbacks=callbacks_list, verbose=0)
           
            # Load the opitimal weights
            model.load_weights("weights.best.hdf5")
            # Compile model (required to make predictions)
            model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])
            print("Created model and loaded weights from file") 
            
            # Prediction
            y_pred = model.predict(X_test)
            R2 = r2_score(Y_test, y_pred)
            
            y_pred_train = model.predict(X_train)
            R2_train = r2_score(Y_train, y_pred_train)
            
            # Save the model
            model_json = model.to_json()
            with open(str(R2)+ "_model.json", "w") as json_file:
                json_file.write(model_json)
            model.save_weights(str(R2)+ "_model.h5")
            para[R2] = (R2, R2_train, bt, lr, dr)

# Extract the best prameters
R2_max = max(para)
bt = para[R2_max][2]
lr = para[R2_max][3]
dr = para[R2_max][4]

#R2_max = 0.3234658252296695

json_file = open(str(R2_max)+'_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(str(R2_max)+"_model.h5")
print("Loaded best_model from disk")

# Prediction
y_pred = loaded_model.predict(X_test)

# The mean squared error
MSE = mean_squared_error(Y_test, y_pred)
RMSE = np.sqrt (MSE)
print("Tesing RMSE: %.3f" % RMSE)
testY_mean = np.mean(Y_test)
RRMSE = 100*RMSE/testY_mean
print("Tesing Relative RMSE: %.3f" % RRMSE)
R2 = r2_score(Y_test, y_pred)
print('Tesing R2: %.3f' % R2)

## Training accuracy metrics
y_pred_train = loaded_model.predict(X_train) 
MSE = mean_squared_error(Y_train, y_pred_train)
RMSE = np.sqrt (MSE)
print("RMSE of Training: %.3f" % RMSE)
trainY_mean = np.mean(Y_train)
RRMSE = 100*RMSE/trainY_mean
print("Relative RMSE of Trainging: %.3f" % RRMSE)
# Explained variance score: 1 is perfect prediction
print('Traing R2: %.3f' % r2_score(Y_train, y_pred_train))

y_pred_all = loaded_model.predict(X)

# Visulization
plt.scatter(Y_test,y_pred)
plt.show()
plt.scatter(Y_train,y_pred_train)
plt.show()
plt.scatter(Y,y_pred_all)
plt.show()

batch_size: 8 learn_rate: 0.0005 dropout_rate: 0.2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                12928     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)      


Epoch 00120: val_loss did not improve from 0.01600

Epoch 00121: val_loss did not improve from 0.01600

Epoch 00122: val_loss did not improve from 0.01600

Epoch 00123: val_loss did not improve from 0.01600

Epoch 00124: val_loss did not improve from 0.01600

Epoch 00125: val_loss did not improve from 0.01600

Epoch 00126: val_loss did not improve from 0.01600

Epoch 00127: val_loss did not improve from 0.01600

Epoch 00128: val_loss did not improve from 0.01600

Epoch 00129: val_loss did not improve from 0.01600

Epoch 00130: val_loss did not improve from 0.01600

Epoch 00131: val_loss did not improve from 0.01600

Epoch 00132: val_loss did not improve from 0.01600

Epoch 00133: val_loss did not improve from 0.01600

Epoch 00134: val_loss did not improve from 0.01600

Epoch 00135: val_loss did not improve from 0.01600

Epoch 00136: val_loss did not improve from 0.01600

Epoch 00137: val_loss did not improve from 0.01600

Epoch 00138: val_loss did not improve from 0.01600

Epoch 00139


Epoch 00276: val_loss did not improve from 0.01438

Epoch 00277: val_loss did not improve from 0.01438

Epoch 00278: val_loss did not improve from 0.01438

Epoch 00279: val_loss did not improve from 0.01438

Epoch 00280: val_loss did not improve from 0.01438

Epoch 00281: val_loss did not improve from 0.01438

Epoch 00282: val_loss did not improve from 0.01438

Epoch 00283: val_loss did not improve from 0.01438

Epoch 00284: val_loss did not improve from 0.01438

Epoch 00285: val_loss did not improve from 0.01438

Epoch 00286: val_loss did not improve from 0.01438

Epoch 00287: val_loss did not improve from 0.01438

Epoch 00288: val_loss did not improve from 0.01438

Epoch 00289: val_loss did not improve from 0.01438

Epoch 00290: val_loss did not improve from 0.01438

Epoch 00291: val_loss did not improve from 0.01438

Epoch 00292: val_loss did not improve from 0.01438

Epoch 00293: val_loss did not improve from 0.01438

Epoch 00294: val_loss did not improve from 0.01438

Epoch 00295


Epoch 00092: val_loss improved from 0.01748 to 0.01577, saving model to weights.best.hdf5

Epoch 00093: val_loss did not improve from 0.01577

Epoch 00094: val_loss did not improve from 0.01577

Epoch 00095: val_loss did not improve from 0.01577

Epoch 00096: val_loss did not improve from 0.01577

Epoch 00097: val_loss did not improve from 0.01577

Epoch 00098: val_loss did not improve from 0.01577

Epoch 00099: val_loss did not improve from 0.01577

Epoch 00100: val_loss did not improve from 0.01577

Epoch 00101: val_loss did not improve from 0.01577

Epoch 00102: val_loss did not improve from 0.01577

Epoch 00103: val_loss did not improve from 0.01577

Epoch 00104: val_loss did not improve from 0.01577

Epoch 00105: val_loss did not improve from 0.01577

Epoch 00106: val_loss did not improve from 0.01577

Epoch 00107: val_loss improved from 0.01577 to 0.01522, saving model to weights.best.hdf5

Epoch 00108: val_loss did not improve from 0.01522

Epoch 00109: val_loss did not improve


Epoch 00248: val_loss did not improve from 0.01473

Epoch 00249: val_loss did not improve from 0.01473

Epoch 00250: val_loss did not improve from 0.01473

Epoch 00251: val_loss did not improve from 0.01473

Epoch 00252: val_loss did not improve from 0.01473

Epoch 00253: val_loss did not improve from 0.01473

Epoch 00254: val_loss did not improve from 0.01473

Epoch 00255: val_loss did not improve from 0.01473

Epoch 00256: val_loss did not improve from 0.01473

Epoch 00257: val_loss did not improve from 0.01473

Epoch 00258: val_loss did not improve from 0.01473

Epoch 00259: val_loss did not improve from 0.01473

Epoch 00260: val_loss did not improve from 0.01473

Epoch 00261: val_loss did not improve from 0.01473

Epoch 00262: val_loss did not improve from 0.01473

Epoch 00263: val_loss did not improve from 0.01473

Epoch 00264: val_loss did not improve from 0.01473

Epoch 00265: val_loss did not improve from 0.01473

Epoch 00266: val_loss did not improve from 0.01473

Epoch 00267

...


Epoch 00080: val_loss did not improve from 0.01667

Epoch 00081: val_loss did not improve from 0.01667

Epoch 00082: val_loss did not improve from 0.01667

Epoch 00083: val_loss did not improve from 0.01667

Epoch 00084: val_loss did not improve from 0.01667

Epoch 00085: val_loss did not improve from 0.01667

Epoch 00086: val_loss did not improve from 0.01667

Epoch 00087: val_loss did not improve from 0.01667

Epoch 00088: val_loss did not improve from 0.01667

Epoch 00089: val_loss did not improve from 0.01667

Epoch 00090: val_loss did not improve from 0.01667

Epoch 00091: val_loss did not improve from 0.01667

Epoch 00092: val_loss did not improve from 0.01667

Epoch 00093: val_loss did not improve from 0.01667

Epoch 00094: val_loss did not improve from 0.01667

Epoch 00095: val_loss did not improve from 0.01667

Epoch 00096: val_loss did not improve from 0.01667

Epoch 00097: val_loss did not improve from 0.01667

Epoch 00098: val_loss did not improve from 0.01667

Epoch 00099


Epoch 00238: val_loss did not improve from 0.01667

Epoch 00239: val_loss did not improve from 0.01667

Epoch 00240: val_loss did not improve from 0.01667

Epoch 00241: val_loss did not improve from 0.01667

Epoch 00242: val_loss did not improve from 0.01667

Epoch 00243: val_loss did not improve from 0.01667

Epoch 00244: val_loss did not improve from 0.01667

Epoch 00245: val_loss did not improve from 0.01667

Epoch 00246: val_loss did not improve from 0.01667

Epoch 00247: val_loss did not improve from 0.01667

Epoch 00248: val_loss did not improve from 0.01667

Epoch 00249: val_loss did not improve from 0.01667

Epoch 00250: val_loss improved from 0.01667 to 0.01659, saving model to weights.best.hdf5

Epoch 00251: val_loss did not improve from 0.01659

Epoch 00252: val_loss improved from 0.01659 to 0.01618, saving model to weights.best.hdf5

Epoch 00253: val_loss did not improve from 0.01618

Epoch 00254: val_loss did not improve from 0.01618

Epoch 00255: val_loss did not improve


Epoch 00060: val_loss did not improve from 0.01708

Epoch 00061: val_loss did not improve from 0.01708

Epoch 00062: val_loss did not improve from 0.01708

Epoch 00063: val_loss did not improve from 0.01708

Epoch 00064: val_loss did not improve from 0.01708

Epoch 00065: val_loss did not improve from 0.01708

Epoch 00066: val_loss did not improve from 0.01708

Epoch 00067: val_loss did not improve from 0.01708

Epoch 00068: val_loss did not improve from 0.01708

Epoch 00069: val_loss did not improve from 0.01708

Epoch 00070: val_loss did not improve from 0.01708

Epoch 00071: val_loss did not improve from 0.01708

Epoch 00072: val_loss did not improve from 0.01708

Epoch 00073: val_loss did not improve from 0.01708

Epoch 00074: val_loss did not improve from 0.01708

Epoch 00075: val_loss did not improve from 0.01708

Epoch 00076: val_loss did not improve from 0.01708

Epoch 00077: val_loss did not improve from 0.01708

Epoch 00078: val_loss did not improve from 0.01708

Epoch 00079


Epoch 00218: val_loss did not improve from 0.01649

Epoch 00219: val_loss did not improve from 0.01649

Epoch 00220: val_loss did not improve from 0.01649

Epoch 00221: val_loss did not improve from 0.01649

Epoch 00222: val_loss did not improve from 0.01649

Epoch 00223: val_loss did not improve from 0.01649

Epoch 00224: val_loss did not improve from 0.01649

Epoch 00225: val_loss did not improve from 0.01649

Epoch 00226: val_loss did not improve from 0.01649

Epoch 00227: val_loss did not improve from 0.01649

Epoch 00228: val_loss did not improve from 0.01649

Epoch 00229: val_loss did not improve from 0.01649

Epoch 00230: val_loss did not improve from 0.01649

Epoch 00231: val_loss did not improve from 0.01649

Epoch 00232: val_loss did not improve from 0.01649

Epoch 00233: val_loss did not improve from 0.01649

Epoch 00234: val_loss did not improve from 0.01649

Epoch 00235: val_loss did not improve from 0.01649

Epoch 00236: val_loss did not improve from 0.01649

Epoch 00237

KeyboardInterrupt: 